<a href="https://colab.research.google.com/github/kaniarasann/Langchain/blob/main/06_Orchestration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [212]:
#!pip install langchain langgraph langchain_groq pydantic

In [213]:
from typing import Annotated, List
from pydantic import BaseModel, Field
import operator
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langgraph.constants import Send
from langgraph.graph import StateGraph, START, END

In [214]:
from langchain_groq import ChatGroq
from google.colab import userdata
groq_key = userdata.get('groq_key')

In [215]:
llm = ChatGroq(api_key=groq_key, model="llama-3.3-70b-versatile",)

In [216]:
class Section(BaseModel):
    name: str = Field(description="Name of the section")
    description: str = Field(description="Description of the section")

class Sections(BaseModel):
    """Container for multiple sections"""
    sections: List[Section] = Field(description="List of sections for the report")

class State(BaseModel):
    topic: str
    sections: List[Section]= []
    completed_sections: Annotated[List[str], operator.add] = []
    final_report: str = ""

class WorkerState(BaseModel):
    section: Section
    completed_sections: Annotated[List[str], operator.add] = []

In [217]:
planner = llm.with_structured_output(Sections)

In [218]:
def orchestrate(state: State):
    prompt = PromptTemplate.from_template(
        "Generate a detailed section plan for a report titled '{topic}'."
    ).format(topic=state.topic)
    plan: Sections = planner.invoke(prompt)
    return {"sections": plan.sections}  # <- LIST, not wrapper

def llm_call(state):                         # ① accept raw dict
    prompt_template = PromptTemplate.from_template(
        "Write a Markdown section called '{name}'. Description: {description}"
    )
    res = prompt_template | llm
    markdown = res.invoke({
        "name": state["section"].name,       # ② dictionary lookup
        "description": state["section"].description
    })
    return {"completed_sections": [markdown.content]}

def assign_workers(state: State):
    return [Send("llm_call", {"section": s}) for s in state.sections]

def synthesizer(state: State):
    body = "\n\n---\n\n".join(state.completed_sections)
    return {"final_report": f"# Report on {state.topic}\n\n{body}"}

In [219]:
# Graph setup
graph = StateGraph(State)
graph.add_node("orchestration", orchestrate)
graph.add_node("llm_call", llm_call)
graph.add_node("synthesizer", synthesizer)
graph.add_edge(START, "orchestration")
graph.add_conditional_edges("orchestration", assign_workers, ["llm_call"])
graph.add_edge("llm_call", "synthesizer")
graph.add_edge("synthesizer", END)



In [220]:
builder = graph.compile()
result = builder.invoke({"topic": "History of Tamils"})
print(result["final_report"])

# Report on History of Tamils

### Introduction to Tamil History
Tamil history is a rich and diverse field of study that spans over two millennia, encompassing the social, cultural, and political developments of the Tamil people. This report aims to provide an overview of the significant events, kingdoms, and cultural achievements that have shaped the course of Tamil history. 

The significance of Tamil history lies in its unique blend of indigenous and external influences, which have contributed to the development of a distinct Tamil identity. From the ancient Tamil kingdoms of the Chera, Chola, and Pandya dynasties to the medieval period of Tamil literature and art, each era has left an indelible mark on the region's culture and society.

Understanding Tamil history is essential for several reasons:
* It provides insight into the evolution of Tamil language, literature, and art, which have had a profound impact on the region's cultural heritage.
* It highlights the significant contri